In [1]:
import torch
from botorch import acquisition, settings
from utils.optimizer import BayesianOptimizer,LGSO
from utils import problems
from matplotlib import pyplot as plt
import argparse
from utils.models import GP_RBF, GP_Cylindrical_Custom

In [2]:
dev = torch.device('cuda')

In [3]:
class ThreeHump():
    def __init__(self,noise:float = 0) -> None:
        self.noise = noise
    def __call__(self,phi:torch.tensor, x:torch.tensor = None):
        y = 2*(phi[:, 0].pow(2)) - 1.05*(phi[:, 0]**4) + phi[:, 0] ** 6 / 6 + phi[:,0] * phi[:,1] + phi[:, 1] ** 2
        y = y.view(-1,1)
        if self.noise:
            y += x
        return y
    def sample_x(self,phi,*args):
        return torch.randn((phi.shape[0],1),device = phi.device)*self.noise

class stochastic_ThreeHump(ThreeHump):
    def __init__(self, n_samples = 1, average_x = True,bounds_1 = (-2,0), bounds_2 = (2,5)) -> None:
        super().__init__(0)
        self.bounds_1 = bounds_1
        self.bounds_2 = bounds_2
        self.n_samples = n_samples
        self.average_x = average_x
        self.std = 0.0
    @staticmethod
    def dif_sigma(x):
        return torch.sigmoid(x-10)-torch.sigmoid(x)
    def sample_x(self,phi):
        P1 = phi[:,0].div(phi.norm(p=2,dim=-1)).view(-1,1)
        mask = torch.rand((phi.size(0),self.n_samples),device=phi.device).le(P1)
        x1 = problems.uniform_sample((phi.size(0),self.n_samples),self.bounds_1,device=phi.device)
        x2 = problems.uniform_sample((phi.size(0),self.n_samples),self.bounds_2,device=phi.device)
        return torch.where(mask, x1, x2)
    def __call__(self, phi:torch.tensor, x:torch.tensor = None):
        h = super().__call__(phi)
        if x is None: x = self.sample_x(phi)
        mu = x*h + torch.randn_like(h)*self.std
        y =  mu+torch.randn_like(h)*self.std
        #y = self.dif_sigma(y)
        if self.average_x: y = y.mean(-1,keepdim=True)
        return y

In [4]:
N_initial_points = 11
phi_range = torch.as_tensor([-10,10],device=dev,dtype=torch.get_default_dtype()).view(2,-1)  
max_iter = 100
dimensions_phi = 2
if phi_range.size(1) != dimensions_phi: phi_range = phi_range.repeat(1,dimensions_phi)
n_samples = 97
problem_fn = stochastic_ThreeHump(n_samples=n_samples,average_x=False)
model = GP_RBF(phi_range,dev)


In [5]:
initial_phi = (phi_range[1]-phi_range[0])*torch.rand(N_initial_points,dimensions_phi,device=dev)+phi_range[0]

In [6]:
acquisition_fn = acquisition.ExpectedImprovement
optimizer = BayesianOptimizer(problem_fn,model,phi_range,acquisition_fn=acquisition_fn,initial_phi = initial_phi,device = dev)

In [7]:
D = (initial_phi,problem_fn(initial_phi))

In [8]:
phi = 2*torch.ones(1,2)
x = problem_fn.sample_x(phi)

In [9]:
h = ThreeHump(0)(phi)
h

tensor([[9.8667]])

In [14]:
mu = x*h + torch.randn_like(h)*0
print(mu)
y =  mu+torch.randn_like(h)*0
stochastic_ThreeHump.dif_sigma(y).mean(-1)

tensor([[ -2.2383,  -9.1515,  -2.8851, -19.1176,  -8.6307,  27.1678, -19.2110,
          44.2639,  -6.8590,  -5.9229,  -6.6539,  38.2610,  44.9255,  30.1246,
         -19.4470,  26.4677, -19.4788, -15.3358, -11.5028,  -9.7686, -13.1213,
          -7.3782, -16.9055,  26.8063,  -5.0972,  24.4636, -11.6987,  47.7261,
         -12.8406,  26.1231,  43.6857, -14.2480,  40.0233, -17.9812, -19.0765,
          26.2534,  22.8018, -18.8756, -19.4916,  -6.0731, -14.3548,  -4.0871,
          22.5666, -19.0798,  40.3231,  30.2458,  -8.8847, -12.3518, -13.8330,
          -5.3468,  21.5670, -16.5899, -14.6724, -14.4513,  40.0976,  32.6720,
          -9.2921,  -8.4856, -14.2515, -18.0402, -13.7731, -10.3156,  -3.3034,
          -4.5832, -18.1865,  -1.6222, -17.0005, -16.2575, -19.4024,  -1.1285,
         -17.5134,  45.7322,  -4.2399, -13.4797,  -2.8892,  -2.2525,  39.8711,
         -18.1479,  -5.1873,  49.0537, -18.8267,  23.4031,  20.2116, -14.2402,
          -7.8966, -18.5174,  36.2821,  42.1890,  -6

tensor([-0.0097])